In [4]:
from pyspark.sql import SparkSession
import findspark
import os
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'

In [5]:
findspark.init()
findspark.find()

'/usr/local/lib/python3.8/dist-packages/pyspark'

In [4]:
spark = SparkSession.builder \
    .appName('Dataframe task 1') \
    .getOrCreate()


23/07/23 15:40:02 WARN Utils: Your hostname, fhmserhtf3u24a7v9748 resolves to a loopback address: 127.0.1.1; using 172.16.0.43 instead (on interface eth0)
23/07/23 15:40:02 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/23 15:40:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark.stop()

NameError: name 'spark' is not defined

In [5]:
data = [
    ('Max', 55),
    ('Yan', 53),
    ('Dmitry', 54),
    ('Ann', 25),
]
cols = ['Name', 'Age']

In [6]:
df = spark.createDataFrame(data, schema=cols)

In [7]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: long (nullable = true)



In [8]:
df.show()

+------+---+
|  Name|Age|
+------+---+
|   Max| 55|
|   Yan| 53|
|Dmitry| 54|
|   Ann| 25|
+------+---+



---

In [45]:
df = spark \
    .read \
    .load('/user/master/data/events/date=2022-05-25', format='json')

In [46]:
df.show(10)

+--------------------+------------+
|               event|  event_type|
+--------------------+------------+
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
|{null, null, 2022...|subscription|
+--------------------+------------+
only showing top 10 rows



In [14]:
!hdfs dfs -ls /user/master/data/snapshots/channels/actual

Found 2 items
-rw-r--r--   3 hdfs hadoop          0 2023-04-05 08:49 /user/master/data/snapshots/channels/actual/_SUCCESS
-rw-r--r--   3 hdfs hadoop    4295876 2023-04-05 08:49 /user/master/data/snapshots/channels/actual/part-00000-beae86b3-af54-4415-8f29-31dd79cfe178-c000.snappy.parquet


---

In [47]:
df = spark.read.parquet('/user/master/data/snapshots/channels/actual')

In [48]:
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- channel_type: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- image_src: string (nullable = true)



In [49]:
df.write \
    .mode('append') \
    .partitionBy('channel_type') \
    .format('parquet') \
    .save('/user/ivanrychko/analytics/test')

In [39]:
!hdfs dfs -rm -r /user/ivanrychko/analytics/test

Deleted /user/ivanrychko/analytics/test


In [50]:
!hdfs dfs -ls /user/ivanrychko/analytics/test/channel_type=channel


Found 2 items
-rw-r--r--   1 ivanrychko ivanrychko    4255579 2023-07-23 16:01 /user/ivanrychko/analytics/test/channel_type=channel/part-00000-4678d4d9-6c1b-419b-8ee8-e6334460f6ca.c000.snappy.parquet
-rw-r--r--   1 ivanrychko ivanrychko    4255579 2023-07-23 15:58 /user/ivanrychko/analytics/test/channel_type=channel/part-00000-7dea7be7-613c-48ea-aa00-e4ffafa8a47c.c000.snappy.parquet


In [52]:
spark.read.parquet('/user/ivanrychko/analytics/test') \
    .select('channel_type') \
    .orderBy('channel_type') \
    .distinct() \
    .show()

+------------+
|channel_type|
+------------+
|     channel|
|       river|
+------------+

